#### Function Calling의 사용
- 이 코드에서 function calling은 모델이 적절한 시점에 외부 함수(get_current_weather)를 호출하도록 유도합니다. 모델이 What is the current weather in {user_location}?와 같은 질문에 응답할 때, 해당 함수가 정의되어 있으면 모델은 그 함수를 호출하여 날씨 정보를 가져오려 합니다.

- function_call="auto" 설정을 통해 GPT 모델은 적절한 함수가 있을 때 자동으로 그 함수를 호출할 수 있습니다. 여기서는 get_current_weather 함수 명세를 통해 사용자가 입력한 위치에 맞는 날씨 정보를 가져올 수 있도록 유도됩니다.

- 만약 모델이 함수 호출을 결정하면, 응답에 function_call이 포함되며, 이때 함수 이름과 그 인자가 함께 반환됩니다. 그런 후에 함수가 호출되고, 결과가 다시 모델에 제공됩니다.

이 과정에서 모델은 단순히 인자를 생성하고, 함수를 호출하는 것은 개발자의 책임입니다. OpenAI의 API는 실제 함수 실행을 수행하지 않으므로, 함수 호출 후 결과를 처리하고 다시 모델에 넘기는 과정을 수동으로 처리하게 됩니다.

In [46]:
!pip install openai -q

openweather 사이트 api로 연결하여 혅날씨를 조회할 수 있는 챗봇

https://home.openweathermap.org/

In [60]:
import openai
import json
import requests

#OpenAI AI 키 설정
from openai import OpenAI
client = OpenAI(api_key="")

#날씨 API 키 설정 (예: openweathermap)
weather_api_key  = '4'

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# 사용자 요청 메시지
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather in Seoul, Korea?"}
]

# GPT-4 모델 호출
response = client.chat.completions.create(
    model = 'gpt-4o-mini-2024-07-18',
    messages = messages,
    functions = [{
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the city to get the weather for",
                },
            },
            "required": ["location"],
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message = response.choices[0].message
messages.append(response_message)
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Seoul, Korea"}', name='get_current_weather'), tool_calls=None)


In [61]:
# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)
    # 함수 호출 및 결과 처리
    if tool_function_name == "get_current_weather":
        location = tool_arguments["location"]
        weather_result = get_current_weather(location)

        # 함수 호출 결과 메시지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_result)
        })

        #모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model = 'gpt-4o-mini-2024-07-18',
            messages = messages,
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    print(response.message.content)

The current weather in Seoul, Korea is clear with a temperature of 35.66°C. The temperature feels like 40.25°C due to humidity and other factors. Here are some additional details:

- **Condition:** Clear sky
- **Humidity:** 45%
- **Wind Speed:** 1.53 m/s coming from the southwest (201°)
- **Visibility:** 10 km
- **Cloud Cover:** 9%

It's a hot day, so stay hydrated if you're out!


전체 코드
*    location 입력 받아서 응답

In [63]:
import openai
import json
import requests

#OpenAI AI 키 설정
from openai import OpenAI
client = OpenAI(api_key="")

#날씨 API 키 설정 (예: openweathermap)
weather_api_key  = '23d2f'

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json

# 사용자 요청 메시지
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Please provide the location to get the current weather information"}
]


# 사용자 입력 받기
user_location = input("Enter the location to get the current weather: ")
messages.append({"role": "user", "content": f"What is the weather in {user_location}"})

# GPT-4 모델 호출
response = client.chat.completions.create(
    model = 'gpt-4o-mini-2024-07-18',
    messages = messages,
    functions = [{
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": f"The name of the city to get the weather for target is {user_location}",
                },
            },
            "required": ["location"],
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message = response.choices[0].message
messages.append(response_message)
print(response_message)


Enter the location to get the current weather: tokyo
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"tokyo"}', name='get_current_weather'), tool_calls=None)


In [64]:
tool_arguments

{'location': 'Seoul, Korea'}

In [65]:
# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)
    # 함수 호출 및 결과 처리
    if tool_function_name == "get_current_weather":
        location = tool_arguments["location"]
        weather_result = get_current_weather(location)

        # 함수 호출 결과 메시지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_result)
        })

        #모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model = 'gpt-4o-mini-2024-07-18',
            messages = messages,
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    print(response.message.content)

TypeError: Object of type method is not JSON serializable

In [58]:
get_current_weather("Seoul")

<bound method Response.json of <Response [200]>>

In [59]:
json.dumps(get_current_weather("Seoul"))

TypeError: Object of type method is not JSON serializable

## How to call functions with model generated arguments

다음 예제에서는 모델에서 생성된 입력을 갖는 함수를 실행하는 방법을 보여주고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현합니다. 단순화를 위해 Chinook 샘플 데이터베이스를 사용하겠습니다 .

참고: 모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서 SQL 생성은 위험할 수 있습니다.

이 코드는 OpenAI의 GPT 모델을 사용하여 SQLite 데이터베이스에서 음악 관련 질문에 대한 답변을 SQL 쿼리로 변환한 후, 해당 쿼리를 실행하여 결과를 반환하는 방식으로 function calling을 구현한 예시입니다. 여기서는 사용자가 앨범 관련 질문을 하면, GPT-4가 질문을 SQL 쿼리로 변환하고, SQLite 데이터베이스를 조회하여 결과를 반환합니다.

In [70]:
import sqlite3
from google.colab import drive

drive.mount('/content/drive')
# sqlite3.connect()를 통해 Google Drive에 있는 Chinook.db  SQLite 데이터 베이스에 연결
conn = sqlite3.connect("/content/drive/MyDrive/kdt_240424/m9_LLM/data/Chinook.db")
print("Opened database successfully")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Opened database successfully


데이터베이스 테이블 및 열 정보 조회
*    get_table_names, get_column_name함수를 통해 데이터베이스의 테이블 및 열 이름을 가져옵니다
*    이 정보는 나중에 GPT 모델이 SQL 쿼리를 생성할 때 사용할 스키마 정보를 제공하는데 활용됩니다.


In [71]:
# 데이터베이스에서 테이블 목록을 추출하는 함수입니다 sqlite.master 테이블에서 "type"이 'table'인 항목들의 이름을 ㅏㄱ져오
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

#PRAGMA table_info(table_name) 명령을 사용하여 테이블의 스키마 정보를 가져요고, 컬럼 이름을 리스트로 변환
def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info({table_name});").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
    return table_dicts

데이터베이스의 테이블과 열 정보를 문자열 형태로 저장하여, 나중에 SQL 쿼리 장성시 참조할 수 있도록 합니다.


In [72]:
database_schema_dict = get_database_info(conn)
database_schema_string  = '\n'.join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)


tools라는 리스트에 ask_database라는 함수 명세를 정의합니다.
- 이 함수는 사용자의 질문에 맞는 SQL 쿼리를 받아 데이터베이스에서 정보를 조회하고, 이를 반환하는 기능을 합니다.
- 함수의 매개변수로 query가 있으며, 이는 SQL 쿼리를 텍스트 형태로 전달받아 실행하는 구조입니다.
- 중요한 점은 database_schema_string이 함수 설명에 포함되어 있어, 모델이 데이터베이스 스키마에 맞는 SQL 쿼리를 생성할 수 있도록 도움을 줍니다.

In [73]:
tools = [
    {
        "type": "function",
        "function":{
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "object",
                        "description": f"""
                        SQL query extracting info to answer the user's question
                        SQL, should be written using this database schema:
                        {database_schema_string}
                        The query should be returned in plain text, not in JSON.
                        """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

## Executing SQL queries



In [74]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

##### Steps to invoke a function call using Chat Completions API:

- 1단계 : 모델이 사용할 도구를 선택하도록 하는 내용으로 모델을 프롬프트합니다. 함수 이름 및 서명과 같은 도구에 대한 설명은 '도구' 목록에 정의되어 API 호출에서 모델에 전달됩니다. 선택한 경우 함수 이름과 매개변수가 응답에 포함됩니다.
- 2단계 : 모델이 함수를 호출하려고 하는지 프로그래밍적으로 확인합니다. 참이면 3단계로 진행합니다.
- 3단계 : 응답에서 함수 이름과 매개변수를 추출하고 매개변수와 함께 함수를 호출합니다. 결과를 메시지에 추가합니다.
- 4단계 : 메시지 목록으로 채팅 완료 API를 호출하여 응답을 가져옵니다.

In [75]:
# 사용자의 요청 메시지를 정의 합니다.
messages = [{
    "role": "system",
    "content": "What is the name of the album with the most tracks?"
}]

# 사용자의 질문에 대한 응답을 생성 tools와 tool_choice 파라미터는 모델이 데이터베이스 쿼리를 인식하고 자동으로 도구를 선택할 수 있도록 돕습니다.
response = client.chat.completions.create(
    model = 'gpt-4o-mini-2024-07-18',
    messages = messages,
    tools = tools,
    tool_choice = 'auto'
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message = response.choices[0].message
messages.append(response_message)

print(response_message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9iOj7ZJOeK8C2aIiEYk4cXLE', function=Function(arguments='{}', name='ask_database'), type='function')])


In [76]:
# 모델 응답에서 도구 호출이 포함되어 있는지 확인하고, 도구 호출이 있다면 도구 호출 10, 함수 이름 및 쿼리 문자열을 추출한다
tool_calls = response_message.tool_calls
if tool_calls:

    # if true the model will return the name of the tool / function to call and the argument(s)
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)["query"]

    # 도구 호출 함수 이름이 'ask_database'인 경우 'ask_database 함수를 호출하여 데이터베이스 쿼리를 실행하고 결과를 messages 리스트에 추가 합니다
    if tool_function_name == "ask_database":
        results  = ask_database(conn, tool_query_string)

        messages.append({
            "role":"tool",
            "tool_call_id":tool_call_id,
            "name":tool_function_name,
            "content":results
        })
        #도구 호출 결과가 포함된 messages 리스트를 사용하여 모델을 다수 호출하고 최종 응답을 출력합니다
        #Note that messages with role 'tool' most be a response to a preceding messages with 'tool_calls
        model_response_with_tool_calls = client.chat.completions.create(
            model = 'gpt-4o-mini-2024-07-18',
            messages = messages,
        ) #  get a new respoonse from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
        # 도구 호출 함수 이름이 "ask_database"가 아닌 경우 오류메시지를 출력하거나 도구호출이 없으면 모델의 응답내용을 바로 출력합니다

    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    print(response.message.content)


KeyError: 'query'

In [81]:
import sqlite3
import json
import openai
import sqlite3
from google.colab import drive

drive.mount('/content/drive')

# OpenAI API 키 설정
from openai import OpenAI
client = OpenAI(api_key="")

# 데이터베이스 연결
conn = sqlite3.connect("/content/drive/MyDrive/kdt_240424/m9_LLM/data/Chinook.db")
print("Opened database successfully")

# 함수 정의
def get_table_names(conn):
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

# 함수 정의
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

# 사용자의 질문에 대한 응답을 생성하는 함수
messages = [{
    "role":"user",
    "content": "What is the name of the album with the most tracks?"
}]

# 'functions' 정의
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                        SQL query extracting info to answer the user's question.
                        SQL should be written using this database schema:
                        {database_schema_string}
                        The query should be returned in plain text, not in JSON.
                    """
                }
            },
            "required": ["query"]
        }
    }
]

# 모델 응답 요청
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    functions=functions,  # 'tools' 대신 'functions' 사용
    function_call="auto"  # 'tool_choice' 대신 'function_call' 사용
)

# 모델의 응답 메시지와 함수 호출 처리
response_message = response.choices[0].message
messages.append(response_message)


# function_call이 있는지 확인
if response_message.function_call:  # 'function_call' 속성을 직접 확인
    # 함수 호출 정보 추출
    function_name = response_message.function_call.name
    function_arguments = json.loads(response_message.function_call.arguments)

    # 'ask_database' 함수 호출
    if function_name == 'ask_database':
        query = function_arguments['query']
        results = ask_database(conn, query)

        messages.append({
            "role": "function",
            "name": function_name,
            "content": results
        })

        # 모델의 최종 응답 요청
        final_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages,
        )

        print(final_response.choices[0].message.content)
else:
    # 함수 호출이 없으면 결과 출력
    print(response_message.content)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Opened database successfully
The album with the most tracks is titled "Greatest Hits."
